In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

In [7]:
#softmax 부드럽게 맥스를 뽑는다? 교수가 뭐라 씨부리는지 잘 모르겠음
#softmax는 (시그마 e의 i승) 분에 (e의 i승)임.전체 합이 1이라서 확률로 볼 수 있음.
z= torch.FloatTensor([1,2,3])
hypothesis= F.softmax(z,dim=0)

print(hypothesis)


tensor([0.0900, 0.2447, 0.6652])


In [18]:
#cross entropy 두개의 확률분포가 얼마나 비슷한지에 관한 수치. 최소화할수록 좋음. 
# 함수가 -시그마P(X)*log Q(x)꼴로 생김. P(x)는 원래 확률, Q(x)는 우리가 가진 확률.
z=torch.rand(3,5,requires_grad=True)#z를 3,5크기의 랜덤한 값을 가진 행렬로 할당
print(z)
hypothesis=F.softmax(z,dim=1)#z를 행에 대해서 소프트맥스
print(hypothesis)
#class=5 samples=3

tensor([[0.4616, 0.4247, 0.8995, 0.9913, 0.3057],
        [0.5389, 0.7175, 0.2800, 0.5472, 0.8061],
        [0.2856, 0.0961, 0.5842, 0.1073, 0.0111]], requires_grad=True)
tensor([[0.1648, 0.1588, 0.2554, 0.2799, 0.1410],
        [0.1893, 0.2263, 0.1461, 0.1909, 0.2473],
        [0.2096, 0.1734, 0.2825, 0.1753, 0.1593]], grad_fn=<SoftmaxBackward0>)


In [23]:
y=torch.randint(5,(3,)).long()#class가 다섯개니까 5개 중에서 3개 무작위 선택해서 정답 만들기
print(y)

tensor([0, 1, 4])


In [27]:
y_one_hot= torch.zeros_like(hypothesis)#hypothesis와 같은 크기의 영으로 찬 행렬 할당
y_one_hot.scatter_(1,y.unsqueeze(1),1)#y.unsqueeze(1)는 y의 크기가 (3,)이므로 (3,1)로 만듬. _(언더바)연산이라 그대로 할당
            #dim=1. 즉, 행에서 y에 따라 1을 뿌려라
print(y_one_hot)

tensor([[1., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 1.]])


In [29]:
cost= (y_one_hot*-torch.log(hypothesis)).sum(dim=1).mean()#비용은 cross entropy 평균임
#이때 y_one_hot과 hypothesis의 크기가 같아 matmul이 아니라 mul이 이루어짐.
print(cost)


tensor(1.7086, grad_fn=<MeanBackward0>)
tensor([[1.8029, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 1.4857, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 1.8373]], grad_fn=<MulBackward0>)


In [30]:
#다양하게 구현하기
#low level
torch.log(F.softmax(z,dim=1))#Q(x)에 소프트맥스 씌우고 로그 취한거
#high level
F.log_softmax(z,dim=1)#상동

#비용 즉, cross entropy loss 다양하게 구현하기
#low level
(y_one_hot*-torch.log(hypothesis)).sum(dim=1).mean()
#high level
F.nll_loss(F.log_softmax(z,dim=1),y)
#사실 이게 제일 편해보임
F.cross_entropy(z,y)

tensor(1.7086, grad_fn=<NllLossBackward0>)

In [31]:
#구현부분
x_train = [[1, 2, 1, 1],
           [2, 1, 3, 2],
           [3, 1, 3, 4],
           [4, 1, 5, 5],
           [1, 7, 5, 5],
           [1, 2, 5, 6],
           [1, 6, 6, 6],
           [1, 7, 7, 7]]#x의 크기8,4
y_train = [2, 2, 2, 1, 1, 1, 0, 0]#y의 크기 8. 그냥 가짓수가 많은 logistic regression이라고 생각하면 될듯.
x_train = torch.FloatTensor(x_train)
y_train = torch.LongTensor(y_train)


In [38]:
# low level cross entropy
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)#w의 크기 4,3. 
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산 (1)
    hypothesis = F.softmax(x_train.matmul(W) + b, dim=1) #크기 8,3
    y_one_hot = torch.zeros_like(hypothesis) #크기 8,3인 원핫벡터 만들기
    y_one_hot.scatter_(1, y_train.unsqueeze(1), 1)#y_train의 크기가 8,1로 바뀜.
    cost = (y_one_hot * -torch.log(F.softmax(hypothesis, dim=1))).sum(dim=1).mean()

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.901535
Epoch  200/1000 Cost: 0.839114
Epoch  300/1000 Cost: 0.807826
Epoch  400/1000 Cost: 0.788472
Epoch  500/1000 Cost: 0.774822
Epoch  600/1000 Cost: 0.764449
Epoch  700/1000 Cost: 0.756191
Epoch  800/1000 Cost: 0.749398
Epoch  900/1000 Cost: 0.743671
Epoch 1000/1000 Cost: 0.738749


In [39]:
#F.cross_entropy 함수 활용
# 모델 초기화
W = torch.zeros((4, 3), requires_grad=True)
b = torch.zeros(1, requires_grad=True)
# optimizer 설정
optimizer = optim.SGD([W, b], lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # Cost 계산 (2)
    z = x_train.matmul(W) + b # or .mm or @
    cost = F.cross_entropy(z, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()

    # 100번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 1.098612
Epoch  100/1000 Cost: 0.761050
Epoch  200/1000 Cost: 0.689991
Epoch  300/1000 Cost: 0.643229
Epoch  400/1000 Cost: 0.604117
Epoch  500/1000 Cost: 0.568256
Epoch  600/1000 Cost: 0.533922
Epoch  700/1000 Cost: 0.500291
Epoch  800/1000 Cost: 0.466908
Epoch  900/1000 Cost: 0.433507
Epoch 1000/1000 Cost: 0.399962


In [40]:
#nn.module 활용
class SoftmaxClassifierModel(nn.Module):
    def __init__(self):
        super().__init__()
        self.linear = nn.Linear(4, 3) # Output이 3!

    def forward(self, x):
        return self.linear(x)
model = SoftmaxClassifierModel()

In [41]:
# optimizer 설정
optimizer = optim.SGD(model.parameters(), lr=0.1)

nb_epochs = 1000
for epoch in range(nb_epochs + 1):

    # H(x) 계산
    prediction = model(x_train)

    # cost 계산
    cost = F.cross_entropy(prediction, y_train)

    # cost로 H(x) 개선
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    # 20번마다 로그 출력
    if epoch % 100 == 0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(
            epoch, nb_epochs, cost.item()
        ))

Epoch    0/1000 Cost: 4.188795
Epoch  100/1000 Cost: 0.639374
Epoch  200/1000 Cost: 0.552567
Epoch  300/1000 Cost: 0.498374
Epoch  400/1000 Cost: 0.454453
Epoch  500/1000 Cost: 0.415336
Epoch  600/1000 Cost: 0.378612
Epoch  700/1000 Cost: 0.342749
Epoch  800/1000 Cost: 0.306616
Epoch  900/1000 Cost: 0.270080
Epoch 1000/1000 Cost: 0.241863
